In [11]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Silences"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(50,
 {'_id': ObjectId('6673df5e3694d27b53a7eb55'),
  'FileId': 'd96d5c7c-b433-4122-bfd6-848ff8fb8cb4',
  'TotalTranscriptionTime': 3324.1600000000017,
  'FileName': '148625.mp4',
  'FolderName': 'Herramientas de Visualización',
  'Duration': 3552.88,
  'SilenceTime': 228.71999999999844})

In [12]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Transcription"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(50,
 {'_id': ObjectId('66728593d002deaab01dd4b3'),
  'FileId': 'a473cf3a-de7f-466f-ba5a-6b2fe51c31b4',
  'FileName': '132379.mp4',
  'FolderName': 'Herramientas de Visualización',
  'Duration': 2187.72,
  'StartDatetime': datetime.datetime(2024, 6, 19, 2, 11, 31, 753000),
  'EndDatetime': datetime.datetime(2024, 6, 19, 2, 15, 31, 93000),
  'SizeMb': 4.4110107421875,
  'Transcription': [{'Id': 0,
    'TranscriptId': '72334268-6bbd-4c43-aeed-e00dc87286b1',
    'StartTime': 0.0,
    'EndTime': 8.78,
    'Text': ' Bueno, pues muchísimas gracias por asistir a esta primera sesión de la asignatura de'},
   {'Id': 1,
    'TranscriptId': '2ae0bdeb-b500-4950-a2f9-f4abc62a7e5a',
    'StartTime': 8.78,
    'EndTime': 16.080000000000002,
    'Text': ' herramientas de visualización del Master de Visualización, perdón, el Master de Visual Analytics.'},
   {'Id': 2,
    'TranscriptId': 'ba5c48e8-72dd-4177-aa11-230b77f10571',
    'StartTime': 16.080000000000002,
    'EndTime': 23.8,
    'Text': ' Como

In [13]:
import pandas as pd
from functions.chroma_utils import *
# Nombre de la base de datos y colección
collection_chroma = init_chroma('Transcription')

for i in data:
    list_transcriptions = i['Transcription']
    df_transcriptions = pd.DataFrame(list_transcriptions)
    list_ids = df_transcriptions['TranscriptId'].tolist()
    list_text = df_transcriptions['Text'].tolist()
    df_metadata = df_transcriptions.drop(["Text", "TranscriptId"], axis=1)
    df_metadata['FileId'] = i['FileId']
    df_metadata['FileName'] = i['FileName']
    df_metadata['FolderName'] = i['FolderName']
    dict_metadata = df_metadata.to_dict(orient='records')
    filter = {
        "$and": [
            {"FolderName": i['FolderName']},
            {"FileName": i['FileName']}
        ]
    }
    delete_from_chroma(collection_chroma, filter)
    save_data_chroma(collection_chroma, list_ids, list_text, dict_metadata)

Total de elementos en la colección: 0
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros eliminados.
Registros elimin

In [2]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Silences"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find({'FileId': '753f4c9e-004e-43a4-95d3-474e41597f08'})

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data), data[0]

(1,
 {'_id': ObjectId('6673d26a3694d27b53a7de5d'),
  'FileId': '753f4c9e-004e-43a4-95d3-474e41597f08',
  'TotalTranscriptionTime': 4987.860000000001,
  'FileName': '89315.mp4',
  'FolderName': 'Visualización Interactiva de la Información',
  'Duration': 5152.8,
  'SilenceTime': 164.9399999999996})

In [6]:
print(documento['Duration'])
for i in documento['Transcription']:
    print(i['Text'])

2748.48
 Perfecto.
 Pues muchas gracias por asistir a una nueva sesión de
 herramientas de visualización.
 Estamos con el primer gran bloque que tiene que ver con
 herramientas de visualización programáticas basadas en
 aplicaciones web.
 En este caso con HTML, JavaScript y D3,
 que es el framework que estamos utilizando y que,
 como hemos comentado en una sesión,
 nació y estaba en la red de la web.
 Y ahora estamos en la red de la web.
 Como hemos comentado en una sesión,
 nació y estaba totalmente orientado a la representación
 gráfica de contenido, ya sea texto,
 gráficos como veremos, incluye animaciones,
 incluye interacción.
 Y lo que vamos a ver hoy, la sesión se va a dividir en dos
 partes.
 Por un lado, la presentación de la actividad,
 que tiene que ver con la actividad grupal que se hace con
 D3.
 Y luego continuaremos con más ejercicios,
 con más elementos que hicimos, o sea,
 sobre el ejercicio que hicimos la semana pasada.
 Les voy a compartir mi pantalla.
 ¿Están viendo

In [3]:
print(documento['Duration'])
for i in documento['Transcription']:
    print(i['Text'])

2748.48
 PERTOR Pero tú puedearing ciertos
 co했던 hab
icating la
 que es el frenguo que estamos utilizando y que como hemos comentado
 de una sesión, nació y está totalmente orientado al representación gráfica
 de contenido, ya se atexo gráficos como veremos, incluyen invaciones,
 incluyen interactión. Y lo que vamos a ver hoy o la sesión se va a dividir
 en dos partes, por lo lado la presentación de interactividad,
 ¿vale?, que tiene que ver con la actividad global de que se ateconte
 de tres y luego continuaremos con más ejerterios con más elementos que hicimos
 o se sobre el ejerterio que hicimos la sesión a la base, ¿vale?,
 te voy a compartir mi pantalla,
 también no vi pantalla,
 perfecto, porque había varias preguntas o bastante preguntas y siempre he dudas
 alrededor de la actividad, ¿no?, como lo primero,
 la actividad simplemente había, creo que es importante explicarlo, ¿no?,
 el objetivo es o supuesto utilizar de tres, pero como siempre digo las herramientas
 de visuete en s

In [9]:
# Realizar la consulta (obtiene todos los documentos)
filter = {
    "FileName":"139008.mp4"
    }
cursor = collection.find(filter)

# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    print(documento)

{'_id': ObjectId('667270cba7af51c467d0276d'), 'FileId': '3913529c-b74b-46bc-bd5b-a8a2dcf3cc2a', 'FileName': '139008.mp4', 'FolderName': 'Gobierno del Dato y Toma de Decisiones', 'Duration': 2893.24, 'SizeMb': 5.9326171875, 'Transcription': [{'Id': 0, 'TranscriptId': 'a644588d-b2d2-46d9-8b9b-5c05dfd88c5e', 'StartTime': 0.0, 'EndTime': 5.0, 'Text': ' y pueden configurar el profavor que se ve bien, se lograré esto,'}, {'Id': 1, 'TranscriptId': '39e16f8d-c906-4b9c-9be2-39b95f552fa7', 'StartTime': 5.0, 'EndTime': 10.0, 'Text': ' compartiendo de espantalla son a 7, la es el punto a la hora,'}, {'Id': 2, 'TranscriptId': 'ff92446e-bb86-4b43-b47b-d75d9cd55d75', 'StartTime': 21.0, 'EndTime': 25.0, 'Text': ' si pueden confirmar, no es si bien la pantalla, se lo se logró de escuale.'}, {'Id': 3, 'TranscriptId': 'd64684d1-3a8a-4fa9-9baf-8e0e72a512c1', 'StartTime': 25.0, 'EndTime': 32.0, 'Text': ' Perfecto, muchas gracias, muchas gracias a Andrés,'}, {'Id': 4, 'TranscriptId': '604adc5a-9c1d-46cd-880

In [10]:
result = collection.delete_many(filter)
print(result.deleted_count, "documentos eliminados")

1 documentos eliminados


In [7]:
# El TranscriptId que estás buscando
transcript_id = '4f9b6965-97d7-4c2a-90f3-015e098f4a53'

# Consulta para encontrar la transcripción específica
resultado = collection.find_one(
    {'Transcription.TranscriptId': transcript_id},
    {
        'Duration':1,
        'FileName':1,
        'FolderName':1,
        'Transcription.$': 1 
    }  # Proyección para devolver solo la transcripción específica
)

# Imprimir el resultado
print(resultado)

{'_id': ObjectId('666f9b50da1ca2495a147728'), 'FileName': '86970.mp4', 'FolderName': 'Análisis e interpretación de datos', 'Duration': 600, 'Transcription': [{'Id': 0, 'TranscriptId': '4f9b6965-97d7-4c2a-90f3-015e098f4a53', 'StartTime': 60.0, 'EndTime': 74.46000000000001, 'Text': ' Bueno, pues podemos empezar. Soy Jara Mesquita, voy a hacer gusto profesor en la'}]}


In [2]:
from functions.chroma_utils import *
collection_chroma = init_chroma('Transcription')
total_count = collection_chroma.count()
print(f"Total de elementos en la colección: {total_count}")
search_chroma(collection_chroma, ['sobre 7 hombres, lo las mujeres'], {'FileId': 'f6a40845-711e-4394-b19b-5cb68bb72017'}, 5)

Total de elementos en la colección: 26831
Total de elementos en la colección: 26831


{'ids': [[]],
 'distances': None,
 'metadatas': [[]],
 'embeddings': [[]],
 'documents': [[]],
 'uris': None,
 'data': None,
 'included': ['embeddings', 'metadatas', 'documents']}

In [14]:
from functions.chroma_utils import *  # Asegúrate de tener la función init_chroma() definida aquí


collection_chroma = init_chroma('Transcription')
total_count = collection_chroma.count()
print(f"Total de elementos en la colección: {total_count}")

# Realizar la búsqueda y almacenar los resultados
results = search_chroma(collection_chroma, ['en que clase se habla de mongo?'], {}, 5)

# Imprimir los resultados
if results['ids']:  # Verificar si hay resultados
    for i, document in enumerate(results['documents']):
        print(f"Resultado {i+1}: {document}")

        # Imprimir metadatos si es necesario (ajusta las claves según tus metadatos)
        print("Metadatos:", results['metadatas'][i])
else:
    print("No se encontraron resultados.")


Total de elementos en la colección: 35622
Total de elementos en la colección: 35622
Resultado 1: [' Yo recomiendo, en el caso de Mongo,', ' de Mongo', ' Yo sé cómo funciona Mongo.', ' que tuvo el mongo instalando.', ' que yo le puedo pasar a MongoDB']
Metadatos: [{'EndTime': 1184.3200000000002, 'FileId': 'bf380a27-82d5-4544-9035-26f327bca5f1', 'FileName': '91686.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 409, 'StartTime': 1182.3200000000002}, {'EndTime': 2053.6800000000003, 'FileId': '97d98b9c-974e-4732-b973-b6174ffd822a', 'FileName': '94621.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 911, 'StartTime': 2051.6800000000003}, {'EndTime': 1479.16, 'FileId': 'bf380a27-82d5-4544-9035-26f327bca5f1', 'FileName': '91686.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 532, 'StartTime': 1477.16}, {'EndTime': 2663.68, 'FileId': '41b98447-b077-4586-bdeb-e953d8f242a7', 'FileName': '86236.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 1032

In [1]:
from functions.mongo_utils import *
collection_mongo = init_mongodb("Metadata", "Transcription")
filter = {
                "FolderName": 'Análisis e Interpretación de Datos',
                "FileName": '88730.mp4'
            }
result = query_mongodb(collection_mongo, filter)
len(result)

1